# Multi-Model Personality Simulation - Study 2

This notebook refactors the original Study 2 BFI-2 to Mini-Marker simulation to work with multiple LLM models using the unified portal.py interface.

## Models to Test
- GPT-4
- GPT-4o  
- Llama-3.3-70B-Instruct
- DeepSeek-V3

## Data Flow
1. Load and preprocess Soto BFI-2 data
2. Apply reverse coding to personality items
3. Map numeric responses to expanded format descriptions
4. Generate personality simulation prompts
5. Run simulations across multiple models
6. Save results for analysis

## Next Steps
After running this notebook, use `study_2_analysis.ipynb` for comprehensive analysis of the results.

In [1]:
import pandas as pd
import sys
from pathlib import Path

# Add shared modules to path
sys.path.append('../shared')

from simulation_utils import (
    SimulationConfig, 
    run_bfi_to_minimarker_simulation,
    retry_failed_participants,
    run_enhanced_bfi_to_minimarker_simulation
)
from schema_bfi2 import expanded_scale
# Updated to use the unified mini_marker_prompt system
from mini_marker_prompt import get_expanded_prompt

## Data Loading and Preprocessing

In [2]:
# Load the Soto BFI-2 dataset
data_path = Path('../../raw_data/Soto_data.xlsx')
if not data_path.exists():
    print(f"Data file not found at {data_path}")
    print("Please ensure the raw_data/Soto_data.xlsx file exists in the project root")
    raise FileNotFoundError(f"Data file not found: {data_path}")

data = pd.read_excel(data_path, sheet_name='data')
print(f"Loaded data shape: {data.shape}")
data.head()

Loaded data shape: (470, 704)


,case_id,age,sex,ethnicity,rel_acquaintance,rel_friend,rel_roommate,rel_boygirlfriend,rel_relative,rel_other,...,tneo_n3_dep,tneo_n4_sel,tneo_n5_imp,tneo_n6_vul,tneo_o1_fan,tneo_o2_aes,tneo_o3_fee,tneo_o4_act,tneo_o5_ide,tneo_o6_val
0,1,27.0,M,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,51.250000,40.181818,64.000000,55.102041,46.639344,46.969697,66.7,57.065217,41.984127,58.039216
1,2,26.0,M,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,69.632353,60.636364,66.272727,65.306122,54.836066,56.439394,51.7,51.630435,51.904762,45.784314
2,3,24.0,F,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,60.441176,74.272727,54.909091,65.306122,75.327869,56.439394,56.7,40.760870,51.904762,58.039216
3,4,33.0,M,3.0,NaN,1.0,NaN,NaN,NaN,NaN,...,67.794118,58.363636,64.000000,52.551020,54.836066,50.757576,36.7,65.217391,63.809524,58.039216
4,5,23.0,F,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,62.279412,67.454545,41.272727,60.204082,50.737705,48.863636,49.2,46.195652,38.015873,38.431373


In [3]:
# Generate column names for TDA (Mini-Marker) and BFI-2 items
tda_columns = [f"tda{i}" for i in range(1, 41)]
sbfi_columns = [f"bfi{i}" for i in range(1, 61)]
selected_columns = tda_columns + sbfi_columns

print(f"Original data shape: {data.shape}")

# Remove rows with missing values in the selected columns
data = data.dropna(subset=selected_columns)
print(f"Data shape after removing missing values: {data.shape}")

Original data shape: (470, 704)
Data shape after removing missing values: (438, 704)


In [4]:
# Reverse coding map for BFI-2 items
reverse_coding_map = {
    'bfi1': 'bfi1', 'bfi2': 'bfi2', 'bfi3': 'bfi3R', 'bfi4': 'bfi4R', 'bfi5': 'bfi5R',
    'bfi6': 'bfi6', 'bfi7': 'bfi7', 'bfi8': 'bfi8R', 'bfi9': 'bfi9R', 'bfi10': 'bfi10',
    'bfi11': 'bfi11R', 'bfi12': 'bfi12R', 'bfi13': 'bfi13', 'bfi14': 'bfi14', 'bfi15': 'bfi15',
    'bfi16': 'bfi16R', 'bfi17': 'bfi17R', 'bfi18': 'bfi18', 'bfi19': 'bfi19', 'bfi20': 'bfi20',
    'bfi21': 'bfi21', 'bfi22': 'bfi22R', 'bfi23': 'bfi23R', 'bfi24': 'bfi24R', 'bfi25': 'bfi25R',
    'bfi26': 'bfi26R', 'bfi27': 'bfi27', 'bfi28': 'bfi28R', 'bfi29': 'bfi29R', 'bfi30': 'bfi30R',
    'bfi31': 'bfi31R', 'bfi32': 'bfi32', 'bfi33': 'bfi33', 'bfi34': 'bfi34', 'bfi35': 'bfi35',
    'bfi36': 'bfi36R', 'bfi37': 'bfi37R', 'bfi38': 'bfi38', 'bfi39': 'bfi39', 'bfi40': 'bfi40',
    'bfi41': 'bfi41', 'bfi42': 'bfi42R', 'bfi43': 'bfi43', 'bfi44': 'bfi44R', 'bfi45': 'bfi45R',
    'bfi46': 'bfi46', 'bfi47': 'bfi47R', 'bfi48': 'bfi48R', 'bfi49': 'bfi49R', 'bfi50': 'bfi50R',
    'bfi51': 'bfi51R', 'bfi52': 'bfi52', 'bfi53': 'bfi53', 'bfi54': 'bfi54', 'bfi55': 'bfi55R',
    'bfi56': 'bfi56', 'bfi57': 'bfi57', 'bfi58': 'bfi58R', 'bfi59': 'bfi59', 'bfi60': 'bfi60'
}

# Apply reverse coding
for key, value in reverse_coding_map.items():
    if value.endswith('R'):  # Reverse coded
        data[key] = 6 - data[key]
    # else: keep original value

print("Reverse coding applied successfully")

Reverse coding applied successfully


In [5]:
# Map numeric values to expanded format descriptions
def map_values(row):
    mapped_row = row.copy()
    for key in expanded_scale:
        if pd.notna(row[key]):  # Check if the value is not NaN
            index = int(row[key]) - 1  # Convert to 0-index
            mapped_row[key] = expanded_scale[key][index]  # Replace with corresponding string
    return mapped_row

# Apply mapping to BFI columns
mapped_data = data[sbfi_columns].apply(map_values, axis=1)

# Create combined BFI-2 description
mapped_data['combined_bfi2'] = mapped_data[[f'bfi{i}' for i in range(1, 61)]].apply(
    lambda row: ' '.join(row), axis=1
)

# Add combined description to original data
data['combined_bfi2'] = mapped_data['combined_bfi2']

print("Personality descriptions created successfully")
print(f"Final data shape: {data.shape}")

Personality descriptions created successfully
Final data shape: (438, 705)


/var/folders/zh/79y5ykz51rxcnjchzb73nfxw0000gn/T/ipykernel_11620/3440998859.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'I am fairly organized.' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  mapped_row[key] = expanded_scale[key][index]  # Replace with corresponding string
/var/folders/zh/79y5ykz51rxcnjchzb73nfxw0000gn/T/ipykernel_11620/3440998859.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'I am fairly disorganized.' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  mapped_row[key] = expanded_scale[key][index]  # Replace with corresponding string
/var/folders/zh/79y5ykz51rxcnjchzb73nfxw0000gn/T/ipykernel_11620/3440998859.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'I am ve

In [6]:
# Preview a personality description
print("Sample personality description:")
print(data.iloc[0]['combined_bfi2'][:500] + "...")

Sample personality description:
I am very outgoing, sociable. I am very compassionate almost always soft-hearted. I am fairly organized. I am somewhat relaxed handle stress somewhat well. I have some artistic interests. I am quite assertive. I am very respectful almost always treat others with respect. I am often lazy. I stay very optimistic after experiencing a setback. I am curious about few things. I often feel excited or eager. I rarely find fault with others. I am very dependable steady. I am fairly moody often have up an...


In [7]:
 # Test prompt generation with first participant
first_participant = data.iloc[0]
sample_prompt = get_expanded_prompt(first_participant['combined_bfi2'])

print("="*80)
print("COMPLETE EXPANDED FORMAT PROMPT SENT TO LLM")
print("="*80)
print(sample_prompt)
print("="*80)
print(f"Prompt length: {len(sample_prompt)} characters")
print(f"Prompt word count: {len(sample_prompt.split())} words")

COMPLETE EXPANDED FORMAT PROMPT SENT TO LLM
### Your Assigned Personality ### 
Based on your detailed personality description below, please rate yourself on the following traits. Consider how each trait applies to you based on the personality profile provided.
I am very outgoing, sociable. I am very compassionate almost always soft-hearted. I am fairly organized. I am somewhat relaxed handle stress somewhat well. I have some artistic interests. I am quite assertive. I am very respectful almost always treat others with respect. I am often lazy. I stay very optimistic after experiencing a setback. I am curious about few things. I often feel excited or eager. I rarely find fault with others. I am very dependable steady. I am fairly moody often have up and down mood swings. I am fairly inventive often find clever ways to do things. I am rarely quiet. I feel a great deal of sympathy for others. I am not particularly systematic rarely keep things in order. I am sometimes tense. I am very muc

## Multi-Model Simulation Configuration

In [8]:
# Configuration for different models and temperatures
models_to_test = ['openai-gpt-3.5-turbo-0125','gpt-4', 'gpt-4o', 'llama', 'deepseek']
temperatures = [1.0]  # Test both deterministic and stochastic responses
batch_size = 25  # Smaller batch size for stability across different APIs

# Create participant data list from DataFrame
participants_data = data.to_dict('records')

print(f"Prepared {len(participants_data)} participants for simulation")

Prepared 438 participants for simulation


## Run Simulations for All Models

In [9]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Run simulations for all model-temperature combinations in parallel
all_results = {}

def run_simulation(model, temperature):
    print(f"\n{'='*60}")
    print(f"Starting simulation: {model} with temperature {temperature}")
    print(f"{'='*60}")
    
    config = SimulationConfig(
        model=model,
        temperature=temperature,
        batch_size=batch_size,
        max_workers=10,
        max_retries=5,  # Enhanced retry logic
        base_wait_time=2.0,
        max_wait_time=60.0
    )
    
    try:
        # Use enhanced simulation with format validation and auto-retry
        # Pass the expanded prompt generator specifically
        results = run_enhanced_bfi_to_minimarker_simulation(
            participants_data=participants_data,
            config=config,
            output_dir="study_2_expanded_results",
            use_enhanced=True,  # Enable enhanced validation
            prompt_generator=get_expanded_prompt  # Use expanded-specific prompts
        )
        
        # Check for any failed participants
        failed_count = sum(1 for r in results if isinstance(r, dict) and 'error' in r)
        if failed_count > 0:
            print(f"Warning: {failed_count} participants failed. Enhanced retry handled most issues.")
        
        print(f"Completed simulation: {model} with temperature {temperature}")
        return (f"{model}_temp{temperature}", results)
    except Exception as e:
        print(f"Error in simulation {model} temp {temperature}: {str(e)}")
        return (f"{model}_temp{temperature}", {"error": str(e)})

# Use ThreadPoolExecutor for parallel execution
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(run_simulation, model, temperature)
        for model in models_to_test
        for temperature in temperatures
    ]
    
    for future in as_completed(futures):
        key, result = future.result()
        all_results[key] = result

print(f"\nCompleted all simulations. Results keys: {list(all_results.keys())}")


Starting simulation: openai-gpt-3.5-turbo-0125 with temperature 1.0
Starting simulation for 438 participants using openai-gpt-3.5-turbo-0125
Temperature: 1.0, Batch size: 25
Processing participants 0 to 24

Starting simulation: gpt-4 with temperature 1.0
Starting simulation for 438 participants using gpt-4
Temperature: 1.0, Batch size: 25
Processing participants 0 to 24

Starting simulation: gpt-4o with temperature 1.0
Starting simulation for 438 participants using gpt-4o
Temperature: 1.0, Batch size: 25
Processing participants 0 to 24

Starting simulation: llama with temperature 1.0
Starting simulation for 438 participants using llama
Temperature: 1.0, Batch size: 25
Processing participants 0 to 24

Starting simulation: deepseek with temperature 1.0
Starting simulation for 438 participants using deepseek
Temperature: 1.0, Batch size: 25
Processing participants 0 to 24
Azure OpenAI error: Connection error.
Error for participant 1 (attempt 1): Connection error.
  Retrying in 1.0 second

## Retry Failed Participants (if any)

In [10]:
# Retry any failed participants
for key, results in all_results.items():
    if isinstance(results, list):
        failed_count = sum(1 for r in results if isinstance(r, dict) and 'error' in r)
        if failed_count > 0:
            print(f"Retrying {failed_count} failed participants for {key}")
            
            # Extract model and temperature from key
            model = key.split('_temp')[0]
            temperature = float(key.split('_temp')[1])
            
            config = SimulationConfig(
                model=model,
                temperature=temperature,
                batch_size=batch_size
            )
            
            updated_results = retry_failed_participants(
                results=results,
                participants_data=participants_data,
                prompt_generator=get_expanded_prompt,  # Use expanded-specific prompt function
                config=config,
                personality_key='combined_bfi2'
            )
            
            all_results[key] = updated_results
            
            # Save updated results
            from simulation_utils import save_simulation_results
            save_simulation_results(updated_results, "study_2_results_I_am", "bfi_to_minimarker_retried", config)

print("Retry process completed")

Retrying 1 failed participants for gpt-4o_temp1.0
Retrying participant 259
JSON parsing failed (attempt #1): No JSON object found in response.
Successfully retried participant 259
Results saved to study_2_results_I_am/bfi_to_minimarker_retried_gpt_4o_temp1_0.json
Retry process completed


## Results Summary

In [11]:
# Analyze results summary
print("Simulation Results Summary:")
print("=" * 50)

for key, results in all_results.items():
    if isinstance(results, list):
        total_participants = len(results)
        successful = sum(1 for r in results if not (isinstance(r, dict) and 'error' in r))
        failed = total_participants - successful
        success_rate = (successful / total_participants) * 100
        
        print(f"{key}:")
        print(f"  Total: {total_participants}, Successful: {successful}, Failed: {failed}")
        print(f"  Success Rate: {success_rate:.1f}%")
        print()
    else:
        print(f"{key}: FAILED - {results.get('error', 'Unknown error')}")
        print()

Simulation Results Summary:
llama_temp1.0:
  Total: 438, Successful: 438, Failed: 0
  Success Rate: 100.0%

openai-gpt-3.5-turbo-0125_temp1.0:
  Total: 438, Successful: 438, Failed: 0
  Success Rate: 100.0%

gpt-4o_temp1.0:
  Total: 438, Successful: 438, Failed: 0
  Success Rate: 100.0%

deepseek_temp1.0:
  Total: 438, Successful: 438, Failed: 0
  Success Rate: 100.0%

gpt-4_temp1.0:
  Total: 438, Successful: 438, Failed: 0
  Success Rate: 100.0%



## Save Preprocessed Data

In [12]:
# Save the preprocessed data for reference
output_path = Path('study_2_expanded_results')  # Updated output directory
output_path.mkdir(exist_ok=True)

data.to_csv(output_path / 'study2_preprocessed_data.csv', index=False)
print(f"Preprocessed data saved to {output_path / 'study2_preprocessed_data.csv'}")

print("\n" + "="*60)
print("ENHANCED SIMULATION COMPLETE!")
print("\nNext steps:")
print("1. Run study_2_analysis.ipynb for comprehensive analysis")
print("2. Results are saved in study_2_expanded_results/ directory")
print("3. Enhanced validation and retry logic handled format issues")
print("="*60)

Preprocessed data saved to study_2_expanded_results/study2_preprocessed_data.csv

ENHANCED SIMULATION COMPLETE!

Next steps:
1. Run study_2_analysis.ipynb for comprehensive analysis
2. Results are saved in study_2_expanded_results/ directory
3. Enhanced validation and retry logic handled format issues
